In [1]:
# Get total number of flights as well as number of flights which are delayed in departure and 
# number of flights delayed in arrival.

# Output should contain 3 columns - FlightCount, DepDelayedCount, ArrDelayedCount

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('Test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 14:46:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = 'airtrafficdata/airtraffic_all/airtraffic-part/flightmonth=200801/part-00252-5cde1303-4ebf-4a12-8fad-f5d9f9c9124a-Copy1.c000.snappy.parquet'


In [4]:
airtraffic = spark.read.parquet(path)

In [5]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [6]:
airtraffic.count()

605659

In [7]:
airtraffic.select('IsArrDelayed').distinct().show()

+------------+
|IsArrDelayed|
+------------+
|         YES|
|          NO|
+------------+



In [8]:
airtraffic.filter((col('IsArrDelayed') == 'YES') & (col('Cancelled') == 0)).count()

280663

In [9]:
airtraffic.filter((col('IsDepDelayed') == 'YES') & (col('Cancelled') == 0)).count()

247905

In [14]:
airtraffic.filter(((col('IsArrDelayed') == 'YES') & (col('Cancelled') == 0)) | 
                 ((col('IsDepDelayed') == 'YES') & (col('Cancelled') == 0))). \
    select('Year', 'Month', 'DayOfMonth', 
           'FlightNum', 'IsDepDelayed', 'IsArrDelayed').show()

+----+-----+----------+---------+------------+------------+
|Year|Month|DayOfMonth|FlightNum|IsDepDelayed|IsArrDelayed|
+----+-----+----------+---------+------------+------------+
|2008|    1|        17|     4977|         YES|         YES|
|2008|    1|        17|     5426|          NO|         YES|
|2008|    1|        18|     5260|         YES|         YES|
|2008|    1|        19|     5276|          NO|         YES|
|2008|    1|        20|     5215|          NO|         YES|
|2008|    1|        20|     5324|          NO|         YES|
|2008|    1|        21|     5056|          NO|         YES|
|2008|    1|        21|     5215|         YES|          NO|
|2008|    1|        21|     5595|         YES|         YES|
|2008|    1|        21|     5610|          NO|         YES|
|2008|    1|        22|     5032|         YES|         YES|
|2008|    1|        22|     5331|          NO|         YES|
|2008|    1|        23|     5033|          NO|         YES|
|2008|    1|        23|     5355|       

In [16]:
airtraffic. \
    filter((col('Cancelled') == 0)). \
    agg(count(lit('1')).alias('FlightCount'),
       sum(when(col('IsDepDelayed')=='YES',1).otherwise(0)).alias('DepDelayedCount'),
        sum(when(col('IsArrDelayed')=='YES',1).otherwise(0)).alias('ArrDelayedCount')). \
    show()
        

+-----------+---------------+---------------+
|FlightCount|DepDelayedCount|ArrDelayedCount|
+-----------+---------------+---------------+
|     588366|         247905|         280663|
+-----------+---------------+---------------+

